# 3. Pre-Processing

In [21]:
import numpy as np
from HMM import unsupervised_HMM

In [16]:
# Load the data
filename = 'data/shakespeare.txt'

with open(filename, 'r') as f:
    content = f.readlines()
    lines = []
    for line in content:
        line = line.strip(',')
        if len(line) > 23:
            words = (line.split('\n')[0].split(' '))
            while '' in words : words.remove('')
            newWords = []
            for w in words:
                w = w.strip(' ,.:;?!\'').lower()
                newWords.append(w)
            lines.append((line.split('\n')[0].split(' ')))

In [19]:
def parse_observations(lines):
    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []
        
        for word in line:
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map

def obs_map_reverser(obs_map):
    obs_map_r = {}

    for key in obs_map:
        obs_map_r[obs_map[key]] = key

    return obs_map_r

In [20]:
obs, obs_map = parse_observations(lines)